# Weeks 3-4: Hidden Markov Models (HMMs)

**Objective:** Understand the structure of a Hidden Markov Model, its components, and how to solve the "decoding" problem (finding the most likely sequence of hidden states) using the Viterbi algorithm.

## Step 1: Build Intuition

In Week 3, we modeled weather using a Markov Chain where we could directly see the states (`Sunny`, `Rainy`). Now, imagine you are a detective locked in a windowless basement. You can't see the weather outside, so the state (`Sunny` or `Rainy`) is **hidden** from you.

However, each day your partner comes downstairs. You can observe whether your partner is carrying an **umbrella**. This observation gives you a clue about the hidden weather state.

- If it's `Rainy` (hidden state), it's highly likely your partner has an `Umbrella` (observation).
- If it's `Sunny` (hidden state), it's unlikely your partner has an `Umbrella`.

A **Hidden Markov Model (HMM)** is perfect for this scenario. It models a system with an underlying Markov process that we cannot observe, but we can see outputs or signals that depend on the hidden state.

## Step 2: Understand the Core Idea

An HMM has two layers:
1.  **The Hidden Layer:** A standard Markov Chain with states and transition probabilities. We can't see this layer.
2.  **The Observation Layer:** A set of possible observations that are probabilistically linked to the hidden states.

To fully define an HMM, we need three key components:

1.  **Transition Probabilities (A):** The probability of moving from one *hidden state* to another. This is the same as the Transition Probability Matrix (TPM) from a regular Markov Chain.
    - `A[i, j] = P(hidden_state_j at t+1 | hidden_state_i at t)`

2.  **Emission Probabilities (B):** The probability of seeing a particular *observation* given that the system is in a specific *hidden state*.
    - `B[i, k] = P(observation_k | hidden_state_i)`

3.  **Initial State Probabilities (π):** The probability of the hidden process starting in each possible state at time `t=0`.
    - `π[i] = P(hidden_state_i at t=0)`

## Step 3: Learn the Definitions and Formulas

An HMM is formally defined by the tuple \(\lambda = (A, B, \pi)\).

For our detective example:
- **Hidden States:** `S = {Sunny, Rainy}`
- **Observations:** `O = {Umbrella, No Umbrella}`

**1. Transition Matrix (A):** (Same as last week)
$$ A = \begin{pmatrix} P(S \to S) & P(S \to R) \\ P(R \to S) & P(R \to R) \end{pmatrix} = \begin{pmatrix} 0.8 & 0.2 \\ 0.4 & 0.6 \end{pmatrix} $$

**2. Emission Matrix (B):**
$$ B = \begin{pmatrix} P(Umbrella|S) & P(No Umbrella|S) \\ P(Umbrella|R) & P(No Umbrella|R) \end{pmatrix} = \begin{pmatrix} 0.1 & 0.9 \\ 0.7 & 0.3 \end{pmatrix} $$

**3. Initial Probabilities (π):** (Let's assume it's equally likely to be sunny or rainy on day 1)
$$ \pi = \begin{pmatrix} P(S) & P(R) \end{pmatrix} = \begin{pmatrix} 0.5 & 0.5 \end{pmatrix} $$

### The Three Fundamental Problems of HMMs
1.  **Likelihood:** Given an HMM \(\lambda\) and a sequence of observations, what is the probability of observing that sequence? (Solved by the **Forward Algorithm**).
2.  **Decoding:** Given an HMM \(\lambda\) and a sequence of observations, what is the most likely sequence of *hidden states* that produced these observations? (Solved by the **Viterbi Algorithm**).
3.  **Learning:** Given a sequence of observations, what are the HMM parameters \(\lambda = (A, B, \pi)\) that best explain the data? (Solved by the **Baum-Welch Algorithm**).

In this notebook, we will focus on the **Decoding** problem, as it is highly intuitive and useful.

## Step 4: Apply and Practice (The Viterbi Algorithm)

**Problem:** Suppose over 3 days, you observe your partner's behavior as: `[Umbrella, No Umbrella, Umbrella]`.

What was the most likely sequence of weather for those three days?

The Viterbi algorithm finds this most probable path using dynamic programming. It builds a table of probabilities, moving one step at a time, and at each step, it only keeps the most likely path to get to that state.

Let's implement it from scratch to understand it.

In [ ]:
import numpy as np

# Define the HMM parameters
# States: 0=Sunny, 1=Rainy
states = ['Sunny', 'Rainy']
pi = np.array([0.5, 0.5])
A = np.array([
    [0.8, 0.2],  # Sunny -> Sunny, Sunny -> Rainy
    [0.4, 0.6]   # Rainy -> Sunny, Rainy -> Rainy
])

# Observations: 0=Umbrella, 1=No Umbrella
observations = ['Umbrella', 'No Umbrella']
B = np.array([
    [0.1, 0.9],  # P(Umbrella|Sunny), P(No Umbrella|Sunny)
    [0.7, 0.3]   # P(Umbrella|Rainy), P(No Umbrella|Rainy)
])

# The sequence of observations we saw
# 0 = Umbrella, 1 = No Umbrella, 0 = Umbrella
obs_seq = [0, 1, 0]

print("HMM Parameters:")
print("Initial Probs (pi):", pi)
print("Transition Matrix (A):\n", A)
print("Emission Matrix (B):\n", B)
print("\nObserved Sequence:", [observations[i] for i in obs_seq])

### Viterbi Algorithm Implementation

In [ ]:
def viterbi(obs_seq, states, pi, A, B):
    """Finds the most likely sequence of hidden states (Viterbi algorithm)."""
    num_states = A.shape[0]
    T = len(obs_seq)
    
    # Viterbi table (stores max probabilities)
    viterbi_table = np.zeros((num_states, T))
    # Backpointer table (stores the path)
    backpointer = np.zeros((num_states, T), dtype=int)
    
    # --- 1. Initialization Step ---
    # Probability of starting in state i AND seeing the first observation
    first_obs = obs_seq[0]
    viterbi_table[:, 0] = pi * B[:, first_obs]
    
    # --- 2. Recursion Step ---
    for t in range(1, T):
        obs = obs_seq[t]
        for s in range(num_states):
            # Probability of transitioning from any previous state to current state s
            prev_probs = viterbi_table[:, t-1] * A[:, s]
            
            # Find the max probability and the state that produced it
            max_prob = np.max(prev_probs)
            best_prev_state = np.argmax(prev_probs)
            
            # Store the probability and backpointer
            # Max prob of path so far * prob of emitting current observation from state s
            viterbi_table[s, t] = max_prob * B[s, obs]
            backpointer[s, t] = best_prev_state
            
    # --- 3. Termination Step ---
    best_path_prob = np.max(viterbi_table[:, -1])
    last_state = np.argmax(viterbi_table[:, -1])
    
    # --- 4. Path Backtracking ---
    best_path = [last_state]
    for t in range(T - 1, 0, -1):
        last_state = backpointer[last_state, t]
        best_path.insert(0, last_state)
        
    return best_path, best_path_prob, viterbi_table

# Run the Viterbi algorithm
best_path_indices, prob, v_table = viterbi(obs_seq, states, pi, A, B)

best_path_names = [states[i] for i in best_path_indices]

print("--- Viterbi Algorithm Results ---")
print("Viterbi Table (log probabilities are often used in practice to avoid underflow):\n", v_table)
print(f"\nProbability of the best path: {prob:.6f}")
print(f"Most likely sequence of hidden states: {best_path_names}")

**Interpretation:**

The algorithm suggests that the most likely weather sequence to produce the observations `[Umbrella, No Umbrella, Umbrella]` is `['Rainy', 'Sunny', 'Rainy']`.

This makes intuitive sense:
- **Day 1 (Umbrella):** Likely `Rainy`.
- **Day 2 (No Umbrella):** The model has to decide. Is it more likely that it stayed `Rainy` and our partner just forgot their umbrella (prob 0.3), or that the weather switched to `Sunny` (transition prob 0.4) and they correctly left the umbrella at home (emission prob 0.9)? The path through `Sunny` is more probable.
- **Day 3 (Umbrella):** The weather likely switched back to `Rainy`.

### Using a Library (`hmmlearn`)

Implementing Viterbi from scratch is great for learning, but for real applications, specialized libraries are more robust and optimized. Let's verify our result using `hmmlearn`.

First, you might need to install it: `pip install hmmlearn`

In [ ]:
from hmmlearn import hmm

# Create an HMM model
# 'c' for categorical observations
model = hmm.CategoricalHMM(n_components=len(states), n_features=len(observations))

# Set the parameters we defined manually
model.startprob_ = pi
model.transmat_ = A
model.emissionprob_ = B

# The observation sequence needs to be a 2D array of shape (n_samples, 1)
obs_seq_2d = np.array(obs_seq).reshape(-1, 1)

# Use the 'decode' method, which implements the Viterbi algorithm
log_prob, path_indices = model.decode(obs_seq_2d)

path_names = [states[i] for i in path_indices]

print("--- hmmlearn Library Results ---")
print(f"Log Probability of the best path: {log_prob:.6f}")
print(f"Most likely sequence of hidden states: {path_names}")

# Compare with our manual calculation
print(f"\nOur manual probability: {prob:.6f}")
print(f"hmmlearn log probability converted to normal probability: {np.exp(log_prob):.6f}")

The results match perfectly! This confirms our understanding and implementation of the Viterbi algorithm.

## Summary & Next Steps

In this notebook, we've made the leap from simple Markov Chains to Hidden Markov Models:
1.  **HMMs** consist of a hidden Markov process and a set of observable emissions.
2.  They are defined by **transition (A)**, **emission (B)**, and **initial (π)** probabilities.
3.  The **Viterbi algorithm** is a powerful tool for decoding the most likely sequence of hidden states given a sequence of observations.
4.  Libraries like `hmmlearn` provide efficient, production-ready implementations of HMM algorithms.

HMMs are used in many fields, including speech recognition (mapping audio signals to words), bioinformatics (finding genes in DNA sequences), and finance (modeling market regimes).

In **Week 5**, we will explore **Branching Processes**, which are used to model population growth, the spread of diseases, or chain reactions.